In [11]:
from docplex.cp.model import *
from docplex.cp.config import get_default
import random

In [12]:
from config import setup
setup()

In [13]:
#UTILS  : 
#Données d'entrée :
#H,F liste des agents
#L[I] préférences de l'agent I

def create_random_problem(n):
    F = []
    H = []
    for i in range(1,n+1):
        F.append('f'+str(i))
        H.append('h'+str(i))

    L= {}
    for f in F:
        pref = list(H)
        random.shuffle(pref)
        L[f]=pref
    for h in H:
        pref = list(F)
        random.shuffle(pref)
        L[h]=pref
    
    return  F, H, L



1. Proposez un modèle de programmation par contrainte pour trouver un mariage stable. Le modèle peut inclure
des contraintes logiques (par exemple if_then). L’utilisation de ce genre de contraintes est accessible dans la
documentation du solveur. Utilisez la recherche en profondeur par défaut ((’DepthFirst’) sans préciser des
heuristiques de branchements. N’affichez pas les traces d’exécutions internes du solveur

In [14]:
def simple_stable_mariage(mdl, F, H, L):
    #Lpos[i,j] postition de j dans L(i) (facilite le traitement)
    def preprocess(H,F,L):
        LPos = {}
        for h in H:
            for f in F:
                LPos[h,f]=L[h].index(f)
                LPos[f,h]=L[f].index(h)
        return LPos
    
    LPos = preprocess(H,F,L) 
    #x[h,f] mariage entre h et f
    x = {}
    for h in H:
        for f in F:
            x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)

    #D[i] cout du marriage de i (Position de son partenaire dans L(i))
    D = {}
    for i in H+F:
        D[i]=mdl.integer_var(name="D_"+i)


    #Ajout des contraintes
    #Contrainte de monogamie :
    for h in H:
        mdl.add(sum([x[h,f] for f in F])==1)
    for f in F:
        mdl.add(sum([x[h,f] for h in H])==1)


    #Valeur de D (Position du partenaire dans le classement) :
    for h in H:
        mdl.add(D[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
    for f in F:
        mdl.add(D[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

    #Couple bloquant =
    #(h,f) est un couple bloquant si 
    #   - le partenaire actuel de h et en plus haute postion de preference que f
    #   - et le partenaire actuel de f et en plus haute postion de preference que h
    for h in H:
        for f in F:
            mdl.add(((D[h]>LPos[h,f]) & (D[f]>LPos[f,h]))==false())

    return x, D


In [15]:
def print_solution(solution,x,D):
    print("\n   SOLUTION :\n")
    for h in H:
        for f in F:
            if(solution[x[h,f]]==1):
                print("Mariage entre ",h," et ",f," (Couts = ",solution[D[h]],solution[D[f]],")")

2. Testez le modèle avec l’exemple ci-dessus (ou un autre exemple) en affichant toutes les solutions. Affichez chaque
solution avec un format que vous jugez compréhensible

In [23]:
mdl = CpoModel(name="Mariages")

F = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
H = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']

L= {'h1': ['f1', 'f3', 'f6', 'f2', 'f4', 'f5'],
    'h2': ['f4', 'f6', 'f1', 'f2', 'f5', 'f3'],
    'h3': ['f1', 'f4', 'f5', 'f3', 'f6', 'f2'],
    'h4': ['f6', 'f5', 'f3', 'f4', 'f2', 'f1'],
    'h5': ['f2', 'f3', 'f1', 'f4', 'f5', 'f6'],
    'h6': ['f3', 'f1', 'f2', 'f6', 'f5', 'f4'],
    'f1': ['h1', 'h5', 'h6', 'h3', 'h2', 'h4'],
    'f2': ['h2', 'h4', 'h6', 'h1', 'h3', 'h5'],
    'f3': ['h4', 'h3', 'h6', 'h2', 'h5', 'h1'],
    'f4': ['h1', 'h3', 'h5', 'h4', 'h2', 'h6'],
    'f5': ['h3', 'h2', 'h6', 'h1', 'h4', 'h5'],
    'f6': ['h5', 'h1', 'h3', 'h6', 'h4', 'h2']}

x, D = simple_stable_mariage(mdl,F,H,L)

lsol = mdl.start_search(SearchType="DepthFirst",trace_log=False)

for solution in lsol:
    print_solution(solution,x,D)



   SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f6  (Couts =  0 4 )
Mariage entre  h5  et  f5  (Couts =  4 5 )
Mariage entre  h6  et  f3  (Couts =  0 2 )

   SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f3  (Couts =  2 0 )
Mariage entre  h5  et  f6  (Couts =  5 0 )
Mariage entre  h6  et  f5  (Couts =  4 2 )

   SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f5  (Couts =  1 4 )
Mariage entre  h5  et  f6  (Couts =  5 0 )
Mariage entre  h6  et  f3  (Couts =  0 2 )


3. Proposez deux stratégies de recherches différentes qui vous semblent très différentes (i.e., deux combinaisons
de type <heuristique de choix de variables + heuristique de choix de valeurs>). On note ces deux stratégies
par S1 et S2

In [28]:
vars = [i for i in x.values()]+[i for i in D.values()]
S1 = mdl.search_phase(vars,varchooser=mdl.select_random_var(), valuechooser=mdl.select_smallest(value()))
S2 = mdl.search_phase(vars,varchooser=mdl.select_random_var(), valuechooser=mdl.select_random_value())


On veut tester S1 et S2 avec une recherche en profondeur sans et avec redémarrage (’DepthFirst’ vs ’Restart’). Voici le protocole de l’étude expérimentale :
* Générez différentes instances randomisées du problème avec différentes tailles (à partir de n = 5). Pour
chaque taille, générez 5 instances.
* Lancez les 4 expérimentations (S1 et S2 avec une recherche en profondeur sans et avec redémarrage) avec
vos jeux de données. Fixez le temps limite à 200s pour chaque exécution 1
.
* Reportez le temps d’exécution et le nombre de nœuds dans des figures en fonction de la taille.
* Quelles sont vos conclusions ?

In [31]:
def run_and_display(S,type,F,H,L):
    mdl = CpoModel(name="Mariages "+type)
    mdl.set_search_phases(S)
    x, D = simple_stable_mariage(mdl,F,H,L)
    solution = mdl.solve(SearchType=type,trace_log=False)
    print_solution(solution,x,D)
    
    
for N in  [5,10,20,30]:
    print("N=",N)
    for I in  range(0,5):
        H, F, L=create_random_problem(N)
        print("S1 + DepthFirst :\n")
        run_and_display(S1,"DepthFirst",F,H,L)
        print("S2 + DepthFirst :\n")
        run_and_display(S2,"DepthFirst",F,H,L)
        print("S1 + Restart :\n")
        run_and_display(S1,"Restart",F,H,L)
        print("S2 + Restart :\n")
        run_and_display(S2,"Restart",F,H,L)

N= 5
S1 + DepthFirst :


   SOLUTION :

Mariage entre  f1  et  h4  (Couts =  2 3 )
Mariage entre  f2  et  h3  (Couts =  1 2 )
Mariage entre  f3  et  h5  (Couts =  1 0 )
Mariage entre  f4  et  h2  (Couts =  1 0 )
Mariage entre  f5  et  h1  (Couts =  0 1 )
S2 + DepthFirst :


   SOLUTION :

Mariage entre  f1  et  h4  (Couts =  2 3 )
Mariage entre  f2  et  h3  (Couts =  1 2 )
Mariage entre  f3  et  h5  (Couts =  1 0 )
Mariage entre  f4  et  h2  (Couts =  1 0 )
Mariage entre  f5  et  h1  (Couts =  0 1 )
S1 + Restart :


   SOLUTION :

Mariage entre  f1  et  h4  (Couts =  2 3 )
Mariage entre  f2  et  h3  (Couts =  1 2 )
Mariage entre  f3  et  h5  (Couts =  1 0 )
Mariage entre  f4  et  h2  (Couts =  1 0 )
Mariage entre  f5  et  h1  (Couts =  0 1 )
S2 + Restart :


   SOLUTION :

Mariage entre  f1  et  h4  (Couts =  2 3 )
Mariage entre  f2  et  h3  (Couts =  1 2 )
Mariage entre  f3  et  h5  (Couts =  1 0 )
Mariage entre  f4  et  h2  (Couts =  1 0 )
Mariage entre  f5  et  h1  (Couts =  0 1 )
S

In [ ]:
def run_and_display(S,type,H,F,L):
    mdl = CpoModel(name="Mariages "+type)
    mdl.set_search_phases(S)
    
for N in  [5,10,20,30]:
    for I in  range(0,5):
        H, F, L=create_random_problem(N)

In [19]:
def add_equity_constraint(mdl,D,H,F):
    DH = mdl.integer_var(name="DH")
    DF = mdl.integer_var(name="DF")

    mdl.add(DH==max([D[h] for h in H]))
    mdl.add(DF==max([D[f] for f in F]))
    mdl.add(minimize(abs(DH-DF)))

    return DH,DF

In [20]:

def print_equity_solution(solution,x,D,DH,DF):
    print_solution(solution,x,D)
    print("\nDF = ",solution[DF])
    print("DH = ",solution[DH])


In [21]:
DH, DF = add_equity_constraint(mdl,D,H,F)
solution = mdl.solve(trace_log=False)
print_equity_solution(solution,x,D,DH,DF)







   SOLUTION :

Mariage entre  h1  et  f1  (Couts =  1 2 )
Mariage entre  h2  et  f18  (Couts =  4 5 )
Mariage entre  h3  et  f20  (Couts =  2 1 )
Mariage entre  h4  et  f19  (Couts =  0 3 )
Mariage entre  h5  et  f4  (Couts =  1 0 )
Mariage entre  h6  et  f8  (Couts =  12 1 )
Mariage entre  h7  et  f9  (Couts =  2 6 )
Mariage entre  h8  et  f16  (Couts =  1 7 )
Mariage entre  h9  et  f6  (Couts =  0 0 )
Mariage entre  h10  et  f13  (Couts =  5 0 )
Mariage entre  h11  et  f7  (Couts =  3 2 )
Mariage entre  h12  et  f12  (Couts =  1 0 )
Mariage entre  h13  et  f11  (Couts =  9 2 )
Mariage entre  h14  et  f15  (Couts =  0 0 )
Mariage entre  h15  et  f3  (Couts =  4 5 )
Mariage entre  h16  et  f2  (Couts =  8 3 )
Mariage entre  h17  et  f5  (Couts =  5 13 )
Mariage entre  h18  et  f17  (Couts =  6 3 )
Mariage entre  h19  et  f10  (Couts =  6 7 )
Mariage entre  h20  et  f14  (Couts =  0 1 )

DF =  13
DH =  12
